### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [4]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [5]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [6]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/keras/src/

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7685 - loss: 0.5158  
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6867 - loss: 0.6043  
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7748 - loss: 0.5328
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7291 - loss: 0.5543 
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7132 - loss: 0.5640
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6021 - loss: 0.6863
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7015 - loss: 0.5900
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6329 - loss: 0.6684 
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7265 - loss: 0.5940 
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7211 - loss: 0.5557
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━

/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8612 - loss: 0.3325
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8616 - loss: 0.3361
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8718 - loss: 0.3076
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8673 - loss: 0.32305
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8676 - loss: 0.3260
Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8670 - loss: 0.319665
Epoch 50/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step - accuracy: 0.8723 - loss: 0.3058
55/84 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step

...

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8843 - loss: 0.2878    
Epoch 47/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step
 33/167 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7664 - loss: 0.5288Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8753 - loss: 0.2976
Epoch 47/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8615 - loss: 0.3137
Epoch 99/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8903 - loss: 0.2588
Epoch 40/100
  1/167 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9062 - loss: 0.2141: 0.2340

/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8347 - loss: 0.3992
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8965 - loss: 0.2371
Epoch 44/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8789 - loss: 0.2885 
Epoch 48/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7950 - loss: 0.4681  
Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8926 - loss: 0.2630 
Epoch 41/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8728 - loss: 0.3060
Epoch 100/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8958 - loss: 0.2431
Epoch 99/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8971 - loss: 0.2400  
Epoch 45/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8861 - loss: 0.2769
Epoch 49/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8476 - loss: 0.3785
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8773 - loss: 0.2911
Epoch 48/100
167/167 ━━━━━━━━━━━━━━━━━━━━

/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/rahul.gupta/rahul/learnings/ai/ANN Classification project/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 809us/step - accuracy: 0.7968 - loss: 0.4901
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - accuracy: 0.8309 - loss: 0.4021
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - accuracy: 0.8411 - loss: 0.3865
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 530us/step - accuracy: 0.8501 - loss: 0.3673
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.8666 - loss: 0.3399
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.8453 - loss: 0.3607
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.8587 - loss: 0.3432
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.8556 - loss: 0.3430
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - accuracy: 0.8635 - loss: 0.3343
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - accuracy: 0.8488 - loss: 0.3516
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - accuracy: 0.8610 - loss: 0.3365
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 